In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from   auxiliar import frft_windowed
import matplotlib.animation as animation
import tempfile
import subprocess
import os

def create_frft_video(signal, sample_rate, complex_part, win_size, num_frames, output_video):
    fig, ax = plt.subplots(figsize=(14, 10))
    plt.ioff()  # Turn off interactive mode
    alpha_values = np.linspace(0, 1, num_frames)
    
    def generate_frame(alpha):
        frft_temp = frft_windowed(signal, win_size, win_size, alpha, complex_part, window_type_input="rectangular", window_type_output="rectangular")
        ax.clear()
        ax.specgram(frft_temp.cpu().numpy(), NFFT=4096, Fs=sample_rate, noverlap=int(4096 * 0.95), window=np.blackman(4096), cmap='YlOrRd_r')
        ax.set_ylim(20, 22050)
        # ax.set_xlim(0, len(signal) / (2*sample_rate))
        ax.set_title(f"Angle = {alpha:.2f}")
        return ax

    def update_frame(i):
        generate_frame(alpha_values[i])
        return ax
    
    ani = animation.FuncAnimation(fig, update_frame, frames=num_frames, blit=False)
    
    temp_video_file = tempfile.NamedTemporaryFile(suffix='.mp4', delete=False).name
    
    ani.save(temp_video_file, writer='ffmpeg', fps=10)
    plt.close(fig)

    # Combine the video with audio using ffmpeg (audio part is optional and requires audio signal)
    if os.path.exists(output_video):
        os.remove(output_video)
    command = [
        'ffmpeg',
        '-i', temp_video_file,
        '-c:v', 'copy',
        output_video
    ]
    subprocess.run(command, check=True)

    # Clean up temporary files
    if os.path.exists(temp_video_file):
        os.remove(temp_video_file)

#device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Example usage
fs = 44100
win_size = 2**16
time = np.arange(0, win_size) / fs
sinusoid = np.sin(2 * np.pi * 10025 * time)
sinusoid_tensor = torch.tensor(sinusoid).to(device)

create_frft_video(sinusoid_tensor, fs, "real_part", win_size, 101, 'frft_video_10k_real.mp4')
create_frft_video(sinusoid_tensor, fs, "full", win_size, 101,      'frft_video_10k_full.mp4')

# Example usage
fs = 44100
win_size = 2**16
time = np.arange(0, win_size) / fs
sinusoid = np.sin(2 * np.pi * 5012.5 * time)
sinusoid_tensor = torch.tensor(sinusoid).to(device)

create_frft_video(sinusoid_tensor, fs, "real_part", win_size, 101, 'frft_video_5k_real.mp4')
create_frft_video(sinusoid_tensor, fs, "full", win_size,      101, 'frft_video_5k_full.mp4')

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena